<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey%20Model%20Mark%20I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
#Import Even-Strength On-Ice Rates Data
skater_EV_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Rates.csv')
#skater_EV_rates_raw.head()

In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_rates = skater_EV_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF/60','GA/60','xGF/60','xGA/60']]
#skater_EV_rates

In [4]:
skater_EV_rates['EV_TOI'] = skater_EV_rates['TOI']*3600
#skater_EV_rates

In [5]:
#Convert rate/60 to rates/s
skater_EV_rates['GF_EV'] = skater_EV_rates['GF/60']/3600
skater_EV_rates['GA_EV'] = skater_EV_rates['GA/60']/3600
skater_EV_rates['xGF_EV'] = skater_EV_rates['xGF/60']/3600
skater_EV_rates['xGA_EV'] = skater_EV_rates['xGA/60']/3600
#skater_EV_rates

In [6]:
#Drop TOI, GF/60, GA/60, xGF/60 and xGA/60 columns
skater_EV_rates = skater_EV_rates.drop(columns = ['TOI','GF/60','GA/60','xGF/60','xGA/60'])
#skater_EV_rates

In [7]:
#Rename columns
skater_EV_rates = skater_EV_rates.rename(columns = {'Position':'Position_EV','GP':'GP_EV'})
#skater_EV_rates

In [8]:
#Import PP On-Ice Rates Data
skater_PP_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/PP%20On-Ice%20Rates.csv')
#skater_PP_rates_raw.head()

In [9]:
#Reduce PP On-Ice Totals Data
skater_PP_rates = skater_PP_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]
#skater_PP_rates

In [10]:
#Convert G±/60 to G/s and TOI to PP_TOI(s)
skater_PP_rates['PP'] = skater_PP_rates['G±/60']/3600
skater_PP_rates['PP_TOI'] = skater_PP_rates['TOI']*3600
#skater_PP_rates

In [11]:
#Drop columns
skater_PP_rates = skater_PP_rates.drop(columns = ['TOI','G±/60'])
#skater_PP_rates

In [12]:
#Rename columns
skater_PP_rates = skater_PP_rates.rename(columns = {'Position':'Position_PP','GP':'GP_PP'})
#skater_PP_rates

In [13]:
#Import SH On-Ice Totals Data
skater_SH_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/SH%20On-Ice%20Rates.csv')
#skater_SH_rates_raw.head()

In [14]:
#Reduce SH On-Ice Totals Data
skater_SH_rates = skater_SH_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]
#skater_SH_rates

In [15]:
#Convert TOI to TOI(s) and G±/60 to G± (s)
skater_SH_rates['SH'] = skater_SH_rates['G±/60']/3600
skater_SH_rates['SH_TOI'] = skater_SH_rates['TOI']*3600
#skater_SH_rates

In [16]:
#Drop columns
skater_SH_rates = skater_SH_rates.drop(columns = ['G±/60','TOI'])
#skater_SH_rates

In [17]:
#Rename columns
skater_SH_rates = skater_SH_rates.rename(columns = {'Position':'Position_SH','GP':'GP_SH'})
#skater_SH_rates

In [18]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
#skater_box_totals_raw.head()

In [19]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2']]
#skater_box_totals.head()

In [20]:
skater_box_totals['TOI_tot'] = skater_box_totals['TOI']*3600
#skater_box_totals

In [21]:
skater_box_totals = skater_box_totals.drop(columns = {'TOI'})
#skater_box_totals

In [22]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine
#pd.options.mode.chained_assignment = None
#forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
#forwards_table.head(10)

#defense_table['PAA'] = defense_table['Points'] - Defense_mean
#defensemen_table.head(10)

#Add dataframes back together and sort again
#concat_frames = [forwards_table, defense_table]
#skater_box_totals = pd.concat(concat_frames)
#skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

In [23]:
#Combine skater EV and PP stats
skater_EV_PP = pd.merge(skater_EV_rates, skater_PP_rates, how="outer", on=['Player','Season'])
#skater_EV_PP

In [24]:
#Drop duplicate columns
skater_EV_PP = skater_EV_PP.drop(columns = ['Team_y','Position_PP','GP_PP'])
#skater_EV_PP

In [25]:
#Merge EV,PP,SH tables together
skater_EV_PP_SH = pd.merge(skater_EV_PP, skater_SH_rates, how="left", on=["Player", "Season"])
#skater_EV_PP_SH

In [26]:
#Clean up data
skater_EV_PP_SH = skater_EV_PP_SH.drop(columns = ['Team','Position_SH','GP_SH'])
#skater_EV_PP_SH

In [27]:
#Merge EV,PP,SH and Points tables together
full_skater_totals = pd.merge(skater_EV_PP_SH, skater_box_totals, how="left", on=["Player", "Season"])
#full_skater_totals.head()

In [28]:
#Replace NaN with 0
full_skater_totals = full_skater_totals.replace(np.NaN, 0)
#full_skater_totals

In [29]:
#Drop Repeated Columns
full_skater_totals = full_skater_totals.drop(columns = ['Team','Position','GP'])
#full_skater_totals.head(10)


In [30]:
full_skater_totals = full_skater_totals.rename(columns = {'Team_x':'Team','Position_EV':'Position'})
#full_skater_totals

In [31]:
#Look at positions to get correct F/D average points
forwards_table = full_skater_totals.loc[full_skater_totals.Position!= "D"]
defense_table = full_skater_totals.loc[full_skater_totals.Position== "D"]

In [32]:
#Calculate Mean Stats of forwards and defensemen
avg_GF_EV_F = forwards_table['GF_EV'].mean()
avg_GA_EV_F = forwards_table['GA_EV'].mean()
avg_xGF_EV_F = forwards_table['xGF_EV'].mean()
avg_xGA_EV_F = forwards_table['xGA_EV'].mean()
avg_PP_F = forwards_table['PP'].mean()
avg_SH_F = forwards_table['SH'].mean()
avg_G_F = forwards_table['G'].mean()
avg_A1_F = forwards_table['A1'].mean()
avg_A2_F = forwards_table['A2'].mean()
#print(Forwards_mean)
avg_GF_EV_D = defense_table['GF_EV'].mean()
avg_GA_EV_D = defense_table['GA_EV'].mean()
avg_xGF_EV_D = defense_table['xGF_EV'].mean()
avg_xGA_EV_D = defense_table['xGA_EV'].mean()
avg_PP_D = defense_table['PP'].mean()
avg_SH_D = defense_table['SH'].mean()
avg_G_D = defense_table['G'].mean()
avg_A1_D = defense_table['A1'].mean()
avg_A2_D = defense_table['A2'].mean()
#print(Defensemen_mean)

In [33]:
#Add impact for all variables to the tables
pd.options.mode.chained_assignment = None
#forwards
forwards_table['GF_Impact'] = (forwards_table['GF_EV']-avg_GF_EV_F)*forwards_table['EV_TOI']
forwards_table['GA_Impact'] = (forwards_table['GA_EV']-avg_GA_EV_F)*forwards_table['EV_TOI']
forwards_table['xGF_Impact'] = (forwards_table['xGF_EV']-avg_xGF_EV_F)*forwards_table['EV_TOI']
forwards_table['xGA_Impact'] = (forwards_table['xGA_EV']-avg_xGA_EV_F)*forwards_table['EV_TOI']
forwards_table['PP_Impact'] = (forwards_table['PP']-avg_PP_F)*forwards_table['PP_TOI']
forwards_table['SH_Impact'] = (forwards_table['SH']-avg_SH_F)*forwards_table['SH_TOI']
forwards_table['G_Impact'] = (forwards_table['G']-avg_G_F)*forwards_table['TOI_tot']
forwards_table['A1_Impact'] = (forwards_table['A1']-avg_A1_F)*forwards_table['TOI_tot']
forwards_table['A2_Impact'] = (forwards_table['A2']-avg_A2_F)*forwards_table['TOI_tot']
#defense
defense_table['GF_Impact'] = (defense_table['GF_EV']-avg_GF_EV_D)*defense_table['EV_TOI']
defense_table['GA_Impact'] = (defense_table['GA_EV']-avg_GA_EV_D)*defense_table['EV_TOI']
defense_table['xGF_Impact'] = (defense_table['xGF_EV']-avg_xGF_EV_D)*defense_table['EV_TOI']
defense_table['xGA_Impact'] = (defense_table['xGA_EV']-avg_xGA_EV_D)*defense_table['EV_TOI']
defense_table['PP_Impact'] = (defense_table['PP']-avg_PP_D)*defense_table['PP_TOI']
defense_table['SH_Impact'] = (defense_table['SH']-avg_SH_D)*defense_table['SH_TOI']
defense_table['G_Impact'] = (defense_table['G']-avg_G_D)*defense_table['TOI_tot']
defense_table['A1_Impact'] = (defense_table['A1']-avg_A1_D)*defense_table['TOI_tot']
defense_table['A2_Impact'] = (defense_table['A2']-avg_A2_D)*defense_table['TOI_tot']

In [34]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Goalie%20Stats.csv')
#goalie_stats_raw.head()

In [35]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','GSAx']]
#goalie_stats.head()

In [36]:
#Add Active / Inactive Column to both data tables
forwards_table['A/I'] = 'Active'
defense_table['A/I'] = 'Active'
goalie_stats['A/I'] = 'Active'

In [37]:
#Write if statement for weighing of season data for skaters
#Forwards Table
forwards_table.loc[forwards_table['Season'] == '21-22', 'Total Impact'] = ((forwards_table['GF_Impact']*0.00294) + (forwards_table['GA_Impact']*-0.00248) + (forwards_table['xGF_Impact']*0.00485) + (forwards_table['xGA_Impact']*-0.00439)
+ (forwards_table['xGF_Impact']*0.00485) + (forwards_table['PP_Impact']*0.00361) + (forwards_table['SH_Impact']*0.00498) + (forwards_table['G_Impact']*0.00749)
+ (forwards_table['A1_Impact']*0.00771) + (forwards_table['A2_Impact']*0.0102))*3
forwards_table.loc[forwards_table['Season'] == '20-21', 'Total Impact'] = ((forwards_table['GF_Impact']*0.00294) + (forwards_table['GA_Impact']*-0.00248) + (forwards_table['xGF_Impact']*0.00485) + (forwards_table['xGA_Impact']*-0.00439)
+ (forwards_table['xGF_Impact']*0.00485) + (forwards_table['PP_Impact']*0.00361) + (forwards_table['SH_Impact']*0.00498) + (forwards_table['G_Impact']*0.00749)
+ (forwards_table['A1_Impact']*0.00771) + (forwards_table['A2_Impact']*0.0102))*2
forwards_table.loc[forwards_table['Season'] == '19-20', 'Total Impact'] = ((forwards_table['GF_Impact']*0.00294) + (forwards_table['GA_Impact']*-0.00248) + (forwards_table['xGF_Impact']*0.00485) + (forwards_table['xGA_Impact']*-0.00439)
+ (forwards_table['xGF_Impact']*0.00485) + (forwards_table['PP_Impact']*0.00361) + (forwards_table['SH_Impact']*0.00498) + (forwards_table['G_Impact']*0.00749)
+ (forwards_table['A1_Impact']*0.00771) + (forwards_table['A2_Impact']*0.0102))*1
forwards_table

#Defense Table
defense_table.loc[defense_table['Season'] == '21-22', 'Total Impact'] = ((defense_table['GF_Impact']*0.00444) + (defense_table['GA_Impact']*-0.00385)
+ (defense_table['xGF_Impact']*0.00736) + (defense_table['xGA_Impact']*-0.00654) + (defense_table['PP_Impact']*0.00833) + (defense_table['SH_Impact']*0.00498)
+ (defense_table['G_Impact']*0.0132) + (defense_table['A1_Impact']*0.00963) + (defense_table['A2_Impact']*0.00918))*3
defense_table.loc[defense_table['Season'] == '20-21', 'Total Impact'] = ((defense_table['GF_Impact']*0.00444) + (defense_table['GA_Impact']*-0.00385)
+ (defense_table['xGF_Impact']*0.00736) + (defense_table['xGA_Impact']*-0.00654) + (defense_table['PP_Impact']*0.00833) + (defense_table['SH_Impact']*0.00498)
+ (defense_table['G_Impact']*0.0132) + (defense_table['A1_Impact']*0.00963) + (defense_table['A2_Impact']*0.00918))*2
defense_table.loc[defense_table['Season'] == '19-20', 'Total Impact'] = ((defense_table['GF_Impact']*0.00444) + (defense_table['GA_Impact']*-0.00385)
+ (defense_table['xGF_Impact']*0.00736) + (defense_table['xGA_Impact']*-0.00654) + (defense_table['PP_Impact']*0.00833) + (defense_table['SH_Impact']*0.00498)
+ (defense_table['G_Impact']*0.0132) + (defense_table['A1_Impact']*0.00963) + (defense_table['A2_Impact']*0.00918))*1
defense_table

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,GA_Impact,xGF_Impact,xGA_Impact,PP_Impact,SH_Impact,G_Impact,A1_Impact,A2_Impact,A/I,Total Impact
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,...,273.983227,165.122527,86.463107,370.223264,-801.705154,9.575132e+06,9.641494e+07,3.744098e+07,Active,1.398580e+06
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,...,-512.924836,100.946289,-240.607959,912.060693,-54.846263,2.443626e+07,-1.851396e+06,-7.506635e+05,Active,5.957004e+05
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,...,-310.311062,1125.744563,-133.125890,1516.519070,27.675103,6.417089e+07,4.056350e+07,1.245321e+08,Active,7.142753e+06
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,...,33.953097,-35.058811,-79.544584,-0.000000,0.000000,-3.541964e+06,-4.966168e+06,-6.755076e+06,Active,-1.565893e+05
5,Adam Boqvist,19-20,CHI,D,41,2030220.0,0.000919,0.000825,0.000764,0.000819,...,223.339651,220.049942,292.223954,206.282102,0.000000,1.746623e+06,-1.402219e+06,-5.357405e+06,Active,-3.962607e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,Zach Whitecloud,20-21,VGK,D,51,2861352.0,0.000603,0.000439,0.000667,0.000569,...,-790.029304,31.947046,-303.483326,-0.000000,-18.077949,-4.149599e+06,-5.178611e+06,-7.754867e+05,Active,-2.235187e+05
2467,Zach Whitecloud,21-22,VGK,D,59,3576132.0,0.000928,0.000675,0.000772,0.000622,...,-143.018064,417.408177,-190.554674,-0.000000,-589.482289,1.888493e+07,9.640749e+06,-1.292726e+07,Active,6.703658e+05
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,...,-929.900712,-234.012712,-353.499178,-0.000000,-369.116796,8.893756e+06,-1.843913e+07,3.921471e+06,Active,-2.416709e+04
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,...,-125.860285,-212.370568,-393.854605,-0.000000,-353.266201,-4.602651e+06,-5.744010e+06,-8.105298e+06,Active,-3.809522e+05


In [42]:
#Sum all impacts from the previous 3 seasons for a player

#Capitals
Caps_forwards = ['Alex Ovechkin','Evgeny Kuznetsov','Connor Brown','Aliaksei Protas','Dylan Strome','Anthony Mantha','Marcus Johansson','Lars Eller','T.J. Oshie','Conor Sheary','Nic Dowd','Garnet Hathaway']
Caps_defense = ['Martin Fehervary','John Carlson','Dmitri Orlov','Nick Jensen','Erik Gustafsson','Trevor Van Riemsdyk']
Caps_goalie = ['Darcy Kuemper']

#Bruins
Bruins_forwards = ['Taylor Hall','Patrice Bergeron','Jake DeBrusk','Pavel Zacha','David Krejci','David Pastrnak','A.J. Greer','Charlie Coyle','Craig Smith','Nick Foligno','Tomas Nosek','Jakub Lauko']
Bruins_defense = ['Hampus Lindholm','Brandon Carlo','Derek Forbort','Jakub Zboril','Mike Reilly','Connor Clifton']
Bruins_goalie = ['Linus Ullmark']

In [ ]:
#Obtain the strength of each team

#Capitals Strength
Capitals_Strength = Caps_forwards['']

In [38]:
#Add total Impact coplumn to forwards, defense and goalie tables
#Forwards Table
#forwards_table['Total Impact'] = (forwards_table['GF_Impact']*0.00294) + (forwards_table['GA_Impact']*-0.00248) + (forwards_table['xGF_Impact']*0.00485) + (forwards_table['xGA_Impact']*-0.00439)
#+ (forwards_table['xGF_Impact']*0.00485) + (forwards_table['PP_Impact']*0.00361) + (forwards_table['SH_Impact']*0.00498) + (forwards_table['G_Impact']*0.00749)
#+ (forwards_table['A1_Impact']*0.00771) + (forwards_table['A2_Impact']*0.0102)
#Defense Table
#defense_table['Total Impact'] = (defense_table['GF_Impact']*0.00444) + (defense_table['GA_Impact']*-0.00385)
#+ (defense_table['xGF_Impact']*0.00736) + (defense_table['xGA_Impact']*-0.00654) + (defense_table['PP_Impact']*0.00833) + (defense_table['SH_Impact']*0.00498)
#+ (defense_table['G_Impact']*0.0132) + (defense_table['A1_Impact']*0.00963) + (defense_table['A2_Impact']*0.00918)
#Goalie Stats
#goalie_stats['Total Impact'] = goalie_stats['GSAx']*0.00622

#forwards_table

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,GA_Impact,xGF_Impact,xGA_Impact,PP_Impact,SH_Impact,G_Impact,A1_Impact,A2_Impact,A/I,Total Impact
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,...,2.229453,25.494204,-35.882065,-0.000000,0.000000,-2.399057e+06,-2.122005e+06,-1.207542e+06,Active,0.321653
8,Adam Brooks,19-20,TOR,C,7,185760.0,0.000950,0.000319,0.000767,0.000653,...,-68.543072,22.284830,-2.545257,-0.000000,0.000000,-2.063184e+06,-1.254097e+06,-9.388766e+05,Active,0.453409
9,Adam Brooks,20-21,TOR,C,11,397368.0,0.000775,0.000303,0.000489,0.000414,...,-153.246527,-62.709465,-100.371479,-0.000000,0.000000,-2.716961e+06,-3.525169e+06,-2.006023e+06,Active,0.663272
10,Adam Brooks,21-22,MTL/VGK/WPG,C,25,768636.0,0.000406,0.000231,0.000572,0.000683,...,-351.940087,-57.247036,12.954360,-0.000000,0.000000,-6.785474e+06,-5.908860e+06,-4.626071e+06,Active,-0.012749
11,Adam Erne,19-20,DET,L,56,2158992.0,0.000242,0.000833,0.000483,0.000658,...,312.841470,-352.709390,-17.587799,-225.588907,0.000000,-2.007068e+07,-1.271366e+07,-1.368341e+07,Active,-4.997354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,Zack MacEwen,20-21,VAN,C,34,1039140.0,0.000589,0.000628,0.000572,0.000631,...,-63.027915,-77.393831,-37.330147,-0.000000,0.000000,-9.965080e+06,-8.814274e+06,-5.015825e+06,Active,-0.240045
2473,Zack MacEwen,21-22,PHI,C,75,2548188.0,0.000475,0.000778,0.000572,0.000864,...,227.670607,-189.785816,503.035894,-0.000000,0.000000,-1.937468e+07,-1.392550e+07,-7.160385e+06,Active,-4.999979
2474,Zack Smith,19-20,CHI,C,50,1850832.0,0.000417,0.000517,0.000469,0.000733,...,-317.908217,-328.072024,123.734966,-0.000000,57.981808,-1.361673e+07,-7.071927e+06,-7.934615e+06,Active,-2.612353
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,...,-312.887239,-304.851557,-329.360143,-0.000000,-965.854758,5.393759e+06,-1.486114e+07,-9.403298e+06,Active,-0.614153


In [ ]:
#Put Players on current teams

#NHL API info
#https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
#https://github.com/dword4/nhlapi
#https://gitlab.com/dword4/nhlapi

#API JSON project example
#https://github.com/danmartin25/World_Weather_Analysis/blob/main/WeatherPy.ipynb

#Test all
#data = requests.get('https://statsapi.web.nhl.com/api/v1/configurations').json()
#data

#Different options
#data = requests.get('https://statsapi.web.nhl.com/api/v1/expands').json()
#data

#Empty Dictionary to Hold Rosters
#active_rosters = []

### NOTE: ONLY RUN REQUEST ONCE, THEN COMMENT OUT. INFO WILL STAY IN VARIABLE
#json_api_data = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster').json()

#Variable where data is stored
#json_api_data

#Devils Roster (Index 0, change indext to 1, 2, etc for other teams rosters)
#json_api_data["teams"][0]

#player_name = json_api_data["teams"][0]
#player_team = 
#player_position = #code


#active_rosters.append{{"Player": player_name,
#                       "Team": player_team,
#                       "Postion": player_position}}




In [ ]:
#Create Roster for each team
Avalanche_forwards = forwards_table.loc[(forwards_table.Player=="Andrew Cogliano")|(forwards_table.Player=="J.T. Compher")|(forwards_table.Player=="Darren Helm")|(forwards_table.Player=="Nazem Kadri")
|(forwards_table.Player=="Gabriel Landeskog")|(forwards_table.Player=="Artturi Lehkonen")|(forwards_table.Player=="Nathan MacKinnon")|(forwards_table.Player=="Alex Newhook")|(forwards_table.Player=="Valeri Nichushkin")
|(forwards_table.Player=="Logan O'Connor")|(forwards_table.Player=="Mikko Rantanen")|(forwards_table.Player=="Nico Sturm")]
Avalanche_defense = defense_table.loc[(defense_table.Player=="Bowen Byram")|(defense_table.Player=="Jack Johnson")
|(defense_table.Player=="Erik Johnson")|(defense_table.Player=="Cale Makar")|(defense_table.Player=="Josh Manson")|(defense_table.Player=="Devon Toews")]
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Player=="Darcy Kuemper")]
#Avalanche_forwards

In [ ]:
#Create Avalanche roster for 2021-2022
forwards_table = Avalanche_forwards.loc[Avalanche_forwards.Season=="21-22"]
Avalanche_defense_2122 = Avalanche_defense.loc[Avalanche_defense.Season=="21-22"]
Avalanche_goalies_2122 = Avalanche_goalie.loc[Avalanche_goalie.Season=="21-22"]
#forwards_table

In [ ]:
#Create strength of roster for Avalanche
Avalanche_Strength_2122 = 0.5 + (defense_table['GF_Impact'].sum()*0.00294) + (Avalanche_forwards_2122['GA_Impact'].sum()*-0.00248) + (Avalanche_forwards_2122['xGF_Impact'].sum()*0.00485) + (Avalanche_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Avalanche_forwards_2122['xGF_Impact'].sum()*0.00485) + (Avalanche_forwards_2122['PP_Impact'].sum()*0.00361) + (Avalanche_forwards_2122['SH_Impact'].sum()*0.00498) + (Avalanche_forwards_2122['G_Impact'].sum()*0.00749)
+ (Avalanche_forwards_2122['A1_Impact'].sum()*0.00771) + (Avalanche_forwards_2122['A2_Impact'].sum()*0.0102) + (defense_table['GF_Impact'].sum()*0.00444) + (Avalanche_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Avalanche_defense_2122['xGF_Impact']*0.00736) + (Avalanche_defense_2122['xGA_Impact']*-0.00654) + (Avalanche_defense_2122['PP_Impact']*0.00833) + (Avalanche_defense_2122['SH_Impact']*0.00498)
+ (Avalanche_defense_2122['G_Impact'].sum()*0.0132) + (Avalanche_defense_2122['A1_Impact'].sum()*0.00963) + (Avalanche_defense_2122['A2_Impact'].sum()*0.00918) + (Avalanche_goalies_2122['GSAx']*0.00622)
Avalanche_Strength_2122

41.364297984805496

In [ ]:
#Create Roster for Lightning
Lightning_forwards = forwards_table.loc[(forwards_table.Player=="Pierre-Edouard Bellemare")|(forwards_table.Player=="Anthony Cirelli")|(forwards_table.Player=="Ross Colton")|(forwards_table.Player=="Brandon Hagel")
|(forwards_table.Player=="Alex Killorn")|(forwards_table.Player=="Nikita Kucherov")|(forwards_table.Player=="Patrick Maroon")|(forwards_table.Player=="Riley Nash")|(forwards_table.Player=="Ondrej Palat")
|(forwards_table.Player=="Nick Paul")|(forwards_table.Player=="Corey Perry")|(forwards_table.Player=="Steven Stamkos")]
Lightning_defense = defense_table.loc[(defense_table.Player=="Zach Bogosian")|(defense_table.Player=="Erik Cernak")
|(defense_table.Player=="Victor Hedman")|(defense_table.Player=="Ryan McDonaugh")|(defense_table.Player=="Jan Rutta")|(defense_table.Player=="Mikhail Sergachev")]
Lightning_goalie = goalie_stats.loc[(goalie_stats.Player=="Andrei Vasilevskiy")]
#Lightning_forwards

In [ ]:
#Create Lightning roster for 2021-2022
Lightning_forwards_2122 = Avalanche_forwards.loc[Avalanche_forwards.Season=="21-22"]
Lightning_defense_2122 = Avalanche_defense.loc[Avalanche_defense.Season=="21-22"]
Lightning_goalies_2122 = Avalanche_goalie.loc[Avalanche_goalie.Season=="21-22"]
#Lightning_forwards_2122

In [ ]:
#Create strength of Lightning roster for 2021-2022 season.
Lightning_Strength_2122 = 0.5 + (Lightning_forwards_2122['GF_Impact'].sum()*0.00294) + (Lightning_forwards_2122['GA_Impact'].sum()*-0.00248) + (Lightning_forwards_2122['xGF_Impact'].sum()*0.00485) + (Lightning_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Lightning_forwards_2122['xGF_Impact'].sum()*0.00485) + (Lightning_forwards_2122['PP_Impact'].sum()*0.00361) + (Lightning_forwards_2122['SH_Impact'].sum()*0.00498) + (Lightning_forwards_2122['G_Impact'].sum()*0.00749)
+ (Lightning_forwards_2122['A1_Impact'].sum()*0.00771) + (Lightning_forwards_2122['A2_Impact'].sum()*0.0102) + (Lightning_defense_2122['GF_Impact'].sum()*0.00444) + (Lightning_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Lightning_defense_2122['xGF_Impact']*0.00736) + (Lightning_defense_2122['xGA_Impact']*-0.00654) + (Lightning_defense_2122['PP_Impact']*0.00833) + (Lightning_defense_2122['SH_Impact']*0.00498)
+ (Lightning_defense_2122['G_Impact'].sum()*0.0132) + (Lightning_defense_2122['A1_Impact'].sum()*0.00963) + (Lightning_defense_2122['A2_Impact'].sum()*0.00918) + (Lightning_goalies_2122['GSAx']*0.00622)
Lightning_Strength_2122

41.364297984805496

In [ ]:
#Game Projection for Game 6 of Stanley Cup Final
Lightning_Probability = Lightning_Strength_2122/(Lightning_Strength_2122+Avalanche_Strength_2122)
Avalanche_Probability = 1-Lightning_Probability
Lightning_Odds = ((1/Lightning_Probability)-1)*100
Avalanche_Odds = ((1/Avalanche_Probability)-1)*100

In [ ]:
print("Lightning Odds:", Lightning_Odds)
print("Avalanche_Odds:", Avalanche_Odds)


Lightning Odds: 100.0
Avalanche_Odds: 100.0


In [ ]:
defense_table.loc[defense_table.Player=="Tj Brodie"]

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,PP_Impact,SH_Impact,G_Impact,A1_Impact,A2_Impact,A/I
2237,Tj Brodie,19-20,CGY,D,64,4086180.0,0.000778,0.000681,0.000731,0.000633,...,453.048211,-140.715103,306.683772,-172.330651,45.709909,-286.536765,3.438310e+06,-7.473886e+06,2.244856e+07,Active
2238,Tj Brodie,20-21,TOR,D,56,3792168.0,0.000856,0.000467,0.000742,0.000581,...,715.396534,-941.692860,326.752223,-360.073182,250.020742,-156.875450,-9.867783e+06,-6.891076e+06,1.200605e+07,Active
2239,Tj Brodie,21-22,TOR,D,82,5474448.0,0.000800,0.000675,0.000772,0.000628,...,728.624448,-218.936258,638.980709,-261.293098,-0.000000,160.227305,4.585123e+06,3.403323e+07,4.250746e+07,Active


In [ ]:
#Make Game projection for Leafs-Capitals from 4/24
#Leafs Roster
Leafs_forwards = forwards_table.loc[(forwards_table.Player=="Nicholas Abruzzese")|(forwards_table.Player=="Colin Blackwell")|(forwards_table.Player=="Kyle Clifford")|(forwards_table.Player=="Pierre Engvall")
|(forwards_table.Player=="David Kampf")|(forwards_table.Player=="Alex Kerfoot")|(forwards_table.Player=="Mitch Marner")|(forwards_table.Player=="Auston Matthews")|(forwards_table.Player=="Ilya Mikheyev")
|(forwards_table.Player=="William Nylander")|(forwards_table.Player=="Wayne Simmonds")|(forwards_table.Player=="Jason Spezza")]
Leafs_defense = defense_table.loc[(defense_table.Player=="Tj Brodie")|(defense_table.Player=="Carl Dahlstrom")
|(defense_table.Player=="Mark Giordano")|(defense_table.Player=="Justin Holl")|(defense_table.Player=="Ilya Lyubushkin")|(defense_table.Player=="Morgan Rielly")]
Leafs_goalie = goalie_stats.loc[(goalie_stats.Player=="Erik Kallgren")]
#Leafs_forwards

#Capitals Roster
Capitals_forwards = forwards_table.loc[(forwards_table.Player=="Nicklas Backstrom")|(forwards_table.Player=="Nic Dowd")|(forwards_table.Player=="Lars Eller")|(forwards_table.Player=="Garnet Hathaway")
|(forwards_table.Player=="Marcus Johansson")|(forwards_table.Player=="Evgeny Kuznetsov")|(forwards_table.Player=="Johan Larsson")|(forwards_table.Player=="Anthony Mantha")|(forwards_table.Player=="T.J. Oshie")
|(forwards_table.Player=="Alex Ovechkin")|(forwards_table.Player=="Conor Sheary")|(forwards_table.Player=="Tom Wilson")]
Capitals_defense = defense_table.loc[(defense_table.Player=="John Carlson")|(defense_table.Player=="Martin Fehervary")
|(defense_table.Player=="Nick Jensen")|(defense_table.Player=="Dmitry Orlov")|(defense_table.Player=="Justin Schultz")|(defense_table.Player=="Trevor Van Riemsdyk")]
Capitals_goalie = goalie_stats.loc[(goalie_stats.Player=="Vitek Vanecek")]
#Capitals_forwards

In [ ]:
#Create Leafs roster for 2021-2022
Leafs_forwards_2122 = Leafs_forwards.loc[Leafs_forwards.Season=="21-22"]
Leafs_defense_2122 = Leafs_defense.loc[Leafs_defense.Season=="21-22"]
Leafs_goalies_2122 = Leafs_goalie.loc[Leafs_goalie.Season=="21-22"]
#Leafs_forwards_2122

#Create Capitals roster for 2021-2022
Capitals_forwards_2122 = Capitals_forwards.loc[Capitals_forwards.Season=="21-22"]
Capitals_defense_2122 = Capitals_defense.loc[Capitals_defense.Season=="21-22"]
Capitals_goalies_2122 = Capitals_goalie.loc[Capitals_goalie.Season=="21-22"]
#Capitals_forwards_2122

In [ ]:
#Create strength of roster for Leafs
Leafs_Strength_2122 = 0.5 + (Leafs_forwards_2122['GF_Impact'].sum()*0.00294) + (Leafs_forwards_2122['GA_Impact'].sum()*-0.00248) + (Leafs_forwards_2122['xGF_Impact'].sum()*0.00485) + (Leafs_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Leafs_forwards_2122['xGF_Impact'].sum()*0.00485) + (Leafs_forwards_2122['PP_Impact'].sum()*0.00361) + (Leafs_forwards_2122['SH_Impact'].sum()*0.00498) + (Leafs_forwards_2122['G_Impact'].sum()*0.00749)
+ (Leafs_forwards_2122['A1_Impact'].sum()*0.00771) + (Leafs_forwards_2122['A2_Impact'].sum()*0.0102) + (Leafs_defense_2122['GF_Impact'].sum()*0.00444) + (Leafs_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Leafs_defense_2122['xGF_Impact']*0.00736) + (Leafs_defense_2122['xGA_Impact']*-0.00654) + (Leafs_defense_2122['PP_Impact']*0.00833) + (Leafs_defense_2122['SH_Impact']*0.00498)
+ (Leafs_defense_2122['G_Impact'].sum()*0.0132) + (Leafs_defense_2122['A1_Impact'].sum()*0.00963) + (Leafs_defense_2122['A2_Impact'].sum()*0.00918) + (Leafs_goalies_2122['GSAx']*0.00622)
#Leafs_Strength_2122

#Create strength of roster for Capitals
Capitals_Strength_2122 = 0.5 + (Capitals_forwards_2122['GF_Impact'].sum()*0.00294) + (Capitals_forwards_2122['GA_Impact'].sum()*-0.00248) + (Capitals_forwards_2122['xGF_Impact'].sum()*0.00485) + (Capitals_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Capitals_forwards_2122['xGF_Impact'].sum()*0.00485) + (Capitals_forwards_2122['PP_Impact'].sum()*0.00361) + (Capitals_forwards_2122['SH_Impact'].sum()*0.00498) + (Capitals_forwards_2122['G_Impact'].sum()*0.00749)
+ (Capitals_forwards_2122['A1_Impact'].sum()*0.00771) + (Capitals_forwards_2122['A2_Impact'].sum()*0.0102) + (Capitals_defense_2122['GF_Impact'].sum()*0.00444) + (Capitals_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Capitals_defense_2122['xGF_Impact']*0.00736) + (Capitals_defense_2122['xGA_Impact']*-0.00654) + (Capitals_defense_2122['PP_Impact']*0.00833) + (Capitals_defense_2122['SH_Impact']*0.00498)
+ (Capitals_defense_2122['G_Impact'].sum()*0.0132) + (Capitals_defense_2122['A1_Impact'].sum()*0.00963) + (Capitals_defense_2122['A2_Impact'].sum()*0.00918) + (Capitals_goalies_2122['GSAx']*0.00622)
#Capitals_Strength_2122

280    6.054376e+06
Name: GSAx, dtype: float64

In [ ]:
#Game Projection for Leafs-Capitals on 4/24
Leafs_Probability = Leafs_Strength_2122/(Leafs_Strength_2122+Capitals_Strength_2122)
Capitals_Probability = 1-Leafs_Probability

#Leafs Odds
if (1/Leafs_Probability) >= 2:
  Leafs_Odds = ((1/Leafs_Probability)-1)*100
elif (1/Leafs_Probability) <2:
  Leafs_Odds = (-100/((1/Leafs_Probability)-1))

if (1/Capitals_Probability) >= 2:
  Capitals_Odds = ((1/Capitals_Probability)-1)*100
elif (1/Capitals_Probability) <2:
  Capitals_Odds = -100/((1/Capitals_Probability)-1)

In [ ]:
print(round(Leafs_Odds), "Leafs Odds", '|', "Capitals_Odds", round(Capitals_Odds))
print("Leafs_Odds:", round(Leafs_Odds))
print("Capitals_Odds:", round(Capitals_Odds))

-335 Leafs Odds | Capitals_Odds 335
Leafs_Odds: -335
Capitals_Odds: 335


In [ ]:
#Create odds for Sharks-Predators on 10/7
#Predators Roster
Predators_forwards = forwards_table.loc[(forwards_table.Player=="Filip Forsberg")|(forwards_table.Player=="Mikael Granlund")|(forwards_table.Player=="Matt Duchene")
|(forwards_table.Player=="Nino Niederreiter")|(forwards_table.Player=="Ryan Johansen")|(forwards_table.Player=="Kiefer Sherwood")
|(forwards_table.Player=="Yakov Trenin")|(forwards_table.Player=="Colton Sissons")|(forwards_table.Player=="Tanner Jeannot")
|(forwards_table.Player=="Cole Smith")|(forwards_table.Player=="Cody Glass")|(forwards_table.Player=="Eeli Tolvanen")]
Predators_defense = defense_table.loc[(defense_table.Player=="Roman Josi")|(defense_table.Player=="Alexandre Carrier")
|(defense_table.Player=="Ryan McDonaugh")|(defense_table.Player=="Mattias Ekholm")
|(defense_table.Player=="Jeremy Lauzon")|(defense_table.Player=="Dante Fabro")]
Predators_goalie = goalie_stats.loc[(goalie_stats.Player=="Juuse Saros")]
#Predators_forwards

#Sharks Roster
Sharks_forwards = forwards_table.loc[(forwards_table.Player=="Timo Meier")|(forwards_table.Player=="Tomas Hertl")|(forwards_table.Player=="Luke Kunin")
|(forwards_table.Player=="Oskar Lindblom")|(forwards_table.Player=="Logan Couture")|(forwards_table.Player=="Kevin Labanc")
|(forwards_table.Player=="Noah Gregor")|(forwards_table.Player=="Nick Bonino")|(forwards_table.Player=="Matthew Nieto")
|(forwards_table.Player=="Steven Lorentz")|(forwards_table.Player=="Nico Sturm")|(forwards_table.Player=="Jonah Gadjovich")]
Sharks_defense = defense_table.loc[(defense_table.Player=="Scott Harrington")|(defense_table.Player=="Erik Karlsson")
|(defense_table.Player=="Marc-Edouard Vlasic")|(defense_table.Player=="Mario Ferraro")
|(defense_table.Player=="Radim Simek")|(defense_table.Player=="Matt Benning")]
Sharks_goalie = goalie_stats.loc[(goalie_stats.Player=="James Reimer")]
#Sharks_forwards

In [ ]:
#Create Predators roster for 2022-2023
Predators_forwards_1920 = Predators_forwards.loc[Predators_forwards.Season=="19-20"]
Predators_defense_1920 = Predators_defense.loc[Predators_defense.Season=="19-20"]
Predators_goalies_1920 = Predators_goalie.loc[Predators_goalie.Season=="19-20"]
Predators_forwards_2021 = Predators_forwards.loc[Predators_forwards.Season=="20-21"]
Predators_defense_2021 = Predators_defense.loc[Predators_defense.Season=="20-21"]
Predators_goalies_2021 = Predators_goalie.loc[Predators_goalie.Season=="20-21"]
Predators_forwards_2122 = Predators_forwards.loc[Predators_forwards.Season=="21-22"]
Predators_defense_2122 = Predators_defense.loc[Predators_defense.Season=="21-22"]
Predators_goalies_2122 = Predators_goalie.loc[Predators_goalie.Season=="21-22"]
#Predators_forwards_2122

#Create Sharks roster for 2022-2023
Sharks_forwards_1920 = Sharks_forwards.loc[Sharks_forwards.Season=="19-20"]
Sharks_defense_1920 = Sharks_defense.loc[Sharks_defense.Season=="19-20"]
Sharks_goalies_1920 = Sharks_goalie.loc[Sharks_goalie.Season=="19-20"]
Sharks_forwards_2021 = Sharks_forwards.loc[Sharks_forwards.Season=="20-21"]
Sharks_defense_2021 = Sharks_defense.loc[Sharks_defense.Season=="20-21"]
Sharks_goalies_2021 = Sharks_goalie.loc[Sharks_goalie.Season=="20-21"]
Sharks_forwards_2122 = Sharks_forwards.loc[Sharks_forwards.Season=="21-22"]
Sharks_defense_2122 = Sharks_defense.loc[Sharks_defense.Season=="21-22"]
Sharks_goalies_2122 = Sharks_goalie.loc[Sharks_goalie.Season=="21-22"]
#Sharks_forwards_2122

In [ ]:
#Create strength of roster for Predators
Predators_Strength_1920 = 0.5 + (Predators_forwards_1920['GF_Impact'].sum()*0.00294) + (Predators_forwards_1920['GA_Impact'].sum()*-0.00248) + (Predators_forwards_1920['xGF_Impact'].sum()*0.00485) + (Predators_forwards_1920['xGA_Impact'].sum()*-0.00439)
+ (Predators_forwards_1920['xGF_Impact'].sum()*0.00485) + (Predators_forwards_1920['PP_Impact'].sum()*0.00361) + (Predators_forwards_1920['SH_Impact'].sum()*0.00498) + (Predators_forwards_1920['G_Impact'].sum()*0.00749)
+ (Predators_forwards_1920['A1_Impact'].sum()*0.00771) + (Predators_forwards_1920['A2_Impact'].sum()*0.0102) + (Predators_defense_1920['GF_Impact'].sum()*0.00444) + (Predators_defense_1920['GA_Impact'].sum()*-0.00385)
+ (Predators_defense_1920['xGF_Impact']*0.00736) + (Predators_defense_1920['xGA_Impact']*-0.00654) + (Predators_defense_1920['PP_Impact']*0.00833) + (Predators_defense_1920['SH_Impact']*0.00498)
+ (Predators_defense_1920['G_Impact'].sum()*0.0132) + (Predators_defense_1920['A1_Impact'].sum()*0.00963) + (Predators_defense_1920['A2_Impact'].sum()*0.00918) + (Predators_goalies_1920['GSAx']*0.00622)
Predators_Strength_2021 = 0.5 + (Predators_forwards_2021['GF_Impact'].sum()*0.00294) + (Predators_forwards_2021['GA_Impact'].sum()*-0.00248) + (Predators_forwards_2021['xGF_Impact'].sum()*0.00485) + (Predators_forwards_2021['xGA_Impact'].sum()*-0.00439)
+ (Predators_forwards_2021['xGF_Impact'].sum()*0.00485) + (Predators_forwards_2021['PP_Impact'].sum()*0.00361) + (Predators_forwards_2021['SH_Impact'].sum()*0.00498) + (Predators_forwards_2021['G_Impact'].sum()*0.00749)
+ (Predators_forwards_2021['A1_Impact'].sum()*0.00771) + (Predators_forwards_2021['A2_Impact'].sum()*0.0102) + (Predators_defense_2021['GF_Impact'].sum()*0.00444) + (Predators_defense_2021['GA_Impact'].sum()*-0.00385)
+ (Predators_defense_2021['xGF_Impact']*0.00736) + (Predators_defense_2021['xGA_Impact']*-0.00654) + (Predators_defense_2021['PP_Impact']*0.00833) + (Predators_defense_2021['SH_Impact']*0.00498)
+ (Predators_defense_2021['G_Impact'].sum()*0.0132) + (Predators_defense_2021['A1_Impact'].sum()*0.00963) + (Predators_defense_2021['A2_Impact'].sum()*0.00918) + (Predators_goalies_2021['GSAx']*0.00622)
Predators_Strength_2122 = 0.5 + (Predators_forwards_2122['GF_Impact'].sum()*0.00294) + (Predators_forwards_2122['GA_Impact'].sum()*-0.00248) + (Predators_forwards_2122['xGF_Impact'].sum()*0.00485) + (Predators_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Predators_forwards_2122['xGF_Impact'].sum()*0.00485) + (Predators_forwards_2122['PP_Impact'].sum()*0.00361) + (Predators_forwards_2122['SH_Impact'].sum()*0.00498) + (Predators_forwards_2122['G_Impact'].sum()*0.00749)
+ (Predators_forwards_2122['A1_Impact'].sum()*0.00771) + (Predators_forwards_2122['A2_Impact'].sum()*0.0102) + (Predators_defense_2122['GF_Impact'].sum()*0.00444) + (Predators_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Predators_defense_2122['xGF_Impact']*0.00736) + (Predators_defense_2122['xGA_Impact']*-0.00654) + (Predators_defense_2122['PP_Impact']*0.00833) + (Predators_defense_2122['SH_Impact']*0.00498)
+ (Predators_defense_2122['G_Impact'].sum()*0.0132) + (Predators_defense_2122['A1_Impact'].sum()*0.00963) + (Predators_defense_2122['A2_Impact'].sum()*0.00918) + (Predators_goalies_2122['GSAx']*0.00622)
Predators_Strength_2223 = Predators_Str

#Create strength of roster for Sharks
Sharks_Strength_1920 = 0.5 + (Sharks_forwards_1920['GF_Impact'].sum()*0.00294) + (Sharks_forwards_1920['GA_Impact'].sum()*-0.00248) + (Sharks_forwards_1920['xGF_Impact'].sum()*0.00485) + (Sharks_forwards_1920['xGA_Impact'].sum()*-0.00439)
+ (Sharks_forwards_1920['xGF_Impact'].sum()*0.00485) + (Sharks_forwards_1920['PP_Impact'].sum()*0.00361) + (Sharks_forwards_1920['SH_Impact'].sum()*0.00498) + (Sharks_forwards_1920['G_Impact'].sum()*0.00749)
+ (Sharks_forwards_1920['A1_Impact'].sum()*0.00771) + (Sharks_forwards_1920['A2_Impact'].sum()*0.0102) + (Sharks_defense_1920['GF_Impact'].sum()*0.00444) + (Sharks_defense_1920['GA_Impact'].sum()*-0.00385)
+ (Sharks_defense_1920['xGF_Impact']*0.00736) + (Sharks_defense_1920['xGA_Impact']*-0.00654) + (Sharks_defense_1920['PP_Impact']*0.00833) + (Sharks_defense_1920['SH_Impact']*0.00498)
+ (Sharks_defense_1920['G_Impact'].sum()*0.0132) + (Sharks_defense_1920['A1_Impact'].sum()*0.00963) + (Sharks_defense_1920['A2_Impact'].sum()*0.00918) + (Sharks_goalies_1920['GSAx']*0.00622)
Sharks_Strength_2021 = 0.5 + (Sharks_forwards_2021['GF_Impact'].sum()*0.00294) + (Sharks_forwards_2021['GA_Impact'].sum()*-0.00248) + (Sharks_forwards_2021['xGF_Impact'].sum()*0.00485) + (Sharks_forwards_2021['xGA_Impact'].sum()*-0.00439)
+ (Sharks_forwards_2021['xGF_Impact'].sum()*0.00485) + (Sharks_forwards_2021['PP_Impact'].sum()*0.00361) + (Sharks_forwards_2021['SH_Impact'].sum()*0.00498) + (Sharks_forwards_2021['G_Impact'].sum()*0.00749)
+ (Sharks_forwards_2021['A1_Impact'].sum()*0.00771) + (Sharks_forwards_2021['A2_Impact'].sum()*0.0102) + (Sharks_defense_2021['GF_Impact'].sum()*0.00444) + (Sharks_defense_2021['GA_Impact'].sum()*-0.00385)
+ (Sharks_defense_2021['xGF_Impact']*0.00736) + (Sharks_defense_2021['xGA_Impact']*-0.00654) + (Sharks_defense_2021['PP_Impact']*0.00833) + (Sharks_defense_2021['SH_Impact']*0.00498)
+ (Sharks_defense_2021['G_Impact'].sum()*0.0132) + (Sharks_defense_2021['A1_Impact'].sum()*0.00963) + (Sharks_defense_2021['A2_Impact'].sum()*0.00918) + (Sharks_goalies_2021['GSAx']*0.00622)
Sharks_Strength_2122 = 0.5 + (Sharks_forwards_2122['GF_Impact'].sum()*0.00294) + (Sharks_forwards_2122['GA_Impact'].sum()*-0.00248) + (Sharks_forwards_2122['xGF_Impact'].sum()*0.00485) + (Sharks_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Sharks_forwards_2122['xGF_Impact'].sum()*0.00485) + (Sharks_forwards_2122['PP_Impact'].sum()*0.00361) + (Sharks_forwards_2122['SH_Impact'].sum()*0.00498) + (Sharks_forwards_2122['G_Impact'].sum()*0.00749)
+ (Sharks_forwards_2122['A1_Impact'].sum()*0.00771) + (Sharks_forwards_2122['A2_Impact'].sum()*0.0102) + (Sharks_defense_2122['GF_Impact'].sum()*0.00444) + (Sharks_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Sharks_defense_2122['xGF_Impact']*0.00736) + (Sharks_defense_2122['xGA_Impact']*-0.00654) + (Sharks_defense_2122['PP_Impact']*0.00833) + (Sharks_defense_2122['SH_Impact']*0.00498)
+ (Sharks_defense_2122['G_Impact'].sum()*0.0132) + (Sharks_defense_2122['A1_Impact'].sum()*0.00963) + (Sharks_defense_2122['A2_Impact'].sum()*0.00918) + (Sharks_goalies_2122['GSAx']*0.00622)
#Sharks_Strength_2122

143    298134.003799
Name: GSAx, dtype: float64

In [ ]:
#Game Projection for Sharks-Predators on 10/7
Predators_Probability = Predators_Strength_2122/(Predators_Strength_2122+Sharks_Strength_2122)
Sharks_Probability = 1-Predators_Probability

#Leafs Odds
if (1/Predators_Probability) >= 2:
  Predators_Odds = ((1/Predators_Probability)-1)*100
elif (1/Predators_Probability) <2:
  Predators_Odds = (-100/((1/Predators_Probability)-1))

if (1/Sharks_Probability) >= 2:
  Sharks_Odds = ((1/Sharks_Probability)-1)*100
elif (1/Sharks_Probability) <2:
  Sharks_Odds = -100/((1/Sharks_Probability)-1)

In [ ]:
Sharks_Probability

-0.5087662451485249

In [ ]:
print("Predators_Odds:", round(Predators_Odds))
print("Sharks_Odds:", round(Sharks_Odds))

Predators_Odds: 297
Sharks_Odds: 34


1. Add adjustment for Home and Away.
2. Adjust for recency of players.
3. Clean up code blocks and refine inputing the data.
4. Bring in team logos for odds and probabilities.
5. Bring the 3 Weighted Seasons in for the start of the 2022-2023 Season.

In [ ]:
#Make a Data Table for skaters on each team
Avalanche_skaters = skater_stats.loc[(skater_stats.Team == 'COL')]
Blackhawks_skaters = skater_stats.loc[(skater_stats.Team == 'CHI')]
Blue_Jackets_skaters = skater_stats.loc[(skater_stats.Team == 'CBJ')]
Blues_skaters = skater_stats.loc[(skater_stats.Team == 'STL')]
Bruins_skaters = skater_stats.loc[(skater_stats.Team == 'BOS')]
Canadiens_skaters = skater_stats.loc[(skater_stats.Team == 'MTL')]
Canucks_skaters = skater_stats.loc[(skater_stats.Team == 'VAN')]
Capitals_skaters = skater_stats.loc[(skater_stats.Team == 'WSH')]
Coyotes_skaters = skater_stats.loc[(skater_stats.Team == 'ARI')]
Devils_skaters = skater_stats.loc[(skater_stats.Team == 'N.J')]
Ducks_skaters = skater_stats.loc[(skater_stats.Team == 'ANA')]
Flames_skaters = skater_stats.loc[(skater_stats.Team == 'CGY')]
Flyers_skaters = skater_stats.loc[(skater_stats.Team == 'PHI')]
Hurricanes_skaters = skater_stats.loc[(skater_stats.Team == 'CAR')]
Islanders_skaters = skater_stats.loc[(skater_stats.Team == 'NYI')]
Jets_skaters = skater_stats.loc[(skater_stats.Team == 'WPG')]
Kings_skaters = skater_stats.loc[(skater_stats.Team == 'L.A')]
Knights_skaters = skater_stats.loc[(skater_stats.Team == 'VGK')]
Kraken_skaters = skater_stats.loc[(skater_stats.Team == 'SEA')]
Leafs_skaters = skater_stats.loc[(skater_stats.Team == 'TOR')]
Lightning_skaters = skater_stats.loc[(skater_stats.Team == 'T.B')]
Oilers_skaters = skater_stats.loc[(skater_stats.Team == 'EDM')]
Panthers_skaters = skater_stats.loc[(skater_stats.Team == 'FLA')]
Penguins_skaters = skater_stats.loc[(skater_stats.Team == 'PIT')]
Predators_skaters = skater_stats.loc[(skater_stats.Team == 'NSH')]
Rangers_skaters = skater_stats.loc[(skater_stats.Team == 'NYR')]
Red_Wings_skaters = skater_stats.loc[(skater_stats.Team == 'DET')]
Sabres_skaters = skater_stats.loc[(skater_stats.Team == 'BUF')]
Senators_skaters = skater_stats.loc[(skater_stats.Team == 'OTT')]
Sharks_skaters = skater_stats.loc[(skater_stats.Team == 'S.J')]
Stars_skaters = skater_stats.loc[(skater_stats.Team == 'DAL')]
Wild_skaters = skater_stats.loc[(skater_stats.Team == 'MIN')]

In [ ]:
#Make a Data for each team of goalies
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Team == 'COL')]
Blackhawks_goalie = goalie_stats.loc[(goalie_stats.Team == 'CHI')]
Blue_Jackets_goalie = goalie_stats.loc[(goalie_stats.Team == 'CBJ')]
Blues_goalie = goalie_stats.loc[(goalie_stats.Team == 'STL')]
Bruins_goalie = goalie_stats.loc[(goalie_stats.Team == 'BOS')]
Canadiens_goalie = goalie_stats.loc[(goalie_stats.Team == 'MTL')]
Canucks_goalie = goalie_stats.loc[(goalie_stats.Team == 'VAN')]
Capitals_goalie = goalie_stats.loc[(goalie_stats.Team == 'WSH')]
Coyotes_goalie = goalie_stats.loc[(goalie_stats.Team == 'ARI')]
Devils_goalie = goalie_stats.loc[(goalie_stats.Team == 'N.J')]
Ducks_goalie = goalie_stats.loc[(goalie_stats.Team == 'ANA')]
Flames_goalie = goalie_stats.loc[(goalie_stats.Team == 'CGY')]
Flyers_goalie = goalie_stats.loc[(goalie_stats.Team == 'PHI')]
Hurricanes_goalie = goalie_stats.loc[(goalie_stats.Team == 'CAR')]
Islanders_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYI')]
Jets_goalie = goalie_stats.loc[(goalie_stats.Team == 'WPG')]
Kings_goalie = goalie_stats.loc[(goalie_stats.Team == 'L.A')]
Knights_goalie = goalie_stats.loc[(goalie_stats.Team == 'VGK')]
Kraken_goalie = goalie_stats.loc[(goalie_stats.Team == 'SEA')]
Leafs_goalie = goalie_stats.loc[(goalie_stats.Team == 'TOR')]
Lightning_goalie = goalie_stats.loc[(goalie_stats.Team == 'T.B')]
Oilers_goalie = goalie_stats.loc[(goalie_stats.Team == 'EDM')]
Panthers_goalie = goalie_stats.loc[(goalie_stats.Team == 'FLA')]
Penguins_goalie = goalie_stats.loc[(goalie_stats.Team == 'PIT')]
Predators_goalie = goalie_stats.loc[(goalie_stats.Team == 'NSH')]
Rangers_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYR')]
Red_Wings_goalie = goalie_stats.loc[(goalie_stats.Team == 'DET')]
Sabres_goalie = goalie_stats.loc[(goalie_stats.Team == 'BUF')]
Senators_goalie = goalie_stats.loc[(goalie_stats.Team == 'OTT')]
Sharks_goalie = goalie_stats.loc[(goalie_stats.Team == 'S.J')]
Stars_goalie = goalie_stats.loc[(goalie_stats.Team == 'DAL')]
Wild_goalie = goalie_stats.loc[(goalie_stats.Team == 'MIN')]